In [27]:
from sklearn import svm

x = [[2,3], [3, 2], [1,1]]
y = [1, 1, -1]

clf = svm.SVC(kernel='linear')
clf.fit(x, y)

print(clf.coef_)
print(clf.intercept_)
print(clf.support_vectors_)

[[0.6668 0.6664]]
[-2.33306667]
[[1. 1.]
 [2. 3.]
 [3. 2.]]


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import dgl
import scipy.sparse as sp

/home/mjyin/miniconda3/envs/recbole-cdr/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
str([0, 1, 2])

'[0, 1, 2]'

In [15]:
rating = F.relu(torch.randn(5, 10)).bool().float()
encoder = nn.Linear(10, 2)
cates = nn.Embedding(3, 2)
decoder = nn.Linear(2, 10)
cores = (F.normalize(decoder.weight) @ F.normalize(cates.weight).T).softmax(-1)

from recbole_cdr.model.layers import Binarize

binary = Binarize.apply

rec_list = []
for _ in range(cates.weight.shape[0]):
    rating_k = rating * cores.T[_:_+1]
    h = F.normalize(encoder(rating_k))
    rec_list.append(decoder(h))
rec = sum(rec_list)
loss = -(F.log_softmax(rec, 1) * rating).sum(1).mean()
print(loss)
loss.backward()

tensor(14.9016, grad_fn=<NegBackward0>)


In [8]:
decoder.weight.shape

torch.Size([10, 2])

In [45]:
data = F.relu(torch.randn(6, 6)).bool().float()
cates = torch.randn(5, 6).softmax(0)
graph_list = []
for _ in range(5):
    cate = cates[_:_+1]
    graph = F.normalize(data * cate)
    graph = graph @ graph.T
    graph_list.append(graph)


In [46]:
graph_list[0]

tensor([[1.000, 0.642, 0.000, 0.000, 0.000, 0.360],
        [0.642, 1.000, 0.000, 0.653, 0.354, 0.562],
        [0.000, 0.000, 1.000, 0.169, 0.887, 0.827],
        [0.000, 0.653, 0.169, 1.000, 0.543, 0.506],
        [0.000, 0.354, 0.887, 0.543, 1.000, 0.933],
        [0.360, 0.562, 0.827, 0.506, 0.933, 1.000]])

In [47]:
graph_list[1]

tensor([[1.000, 0.857, 0.000, 0.000, 0.000, 0.377],
        [0.857, 1.000, 0.000, 0.480, 0.126, 0.440],
        [0.000, 0.000, 1.000, 0.036, 0.970, 0.898],
        [0.000, 0.480, 0.036, 1.000, 0.263, 0.243],
        [0.000, 0.126, 0.970, 0.263, 1.000, 0.926],
        [0.377, 0.440, 0.898, 0.243, 0.926, 1.000]])

In [33]:
def off_diagonal(x):
    b, n, m = x.shape
    assert n == m
    return x.flatten(-2, -1)[:, :-1].view(b, n - 1, n + 1)[:, :, 1:].flatten()
data = torch.randn(2, 3, 3)
print(data)
print(off_diagonal(data))

tensor([[[-1.973, -1.418, -1.368],
         [-0.911, -0.895, -0.260],
         [-1.073,  1.095, -0.585]],

        [[-1.147,  0.992,  0.478],
         [ 0.970,  0.976,  0.422],
         [-0.661,  0.102, -0.968]]])
tensor([-1.418, -1.368, -0.911, -0.260, -1.073,  1.095,  0.992,  0.478,  0.970,
         0.422, -0.661,  0.102])


In [26]:
data.flatten()[:-1].view(2, 4)

tensor([[-0.073,  1.089,  0.627,  0.943],
        [ 0.607, -0.264,  1.133, -0.484]])

In [18]:
(data @ data.transpose(1, 2))

tensor([[[ 1.000, -0.609,  0.171],
         [-0.609,  1.000, -0.231],
         [ 0.171, -0.231,  1.000]],

        [[ 1.000,  0.631, -0.186],
         [ 0.631,  1.000,  0.415],
         [-0.186,  0.415,  1.000]],

        [[ 1.000,  0.354, -0.209],
         [ 0.354,  1.000, -0.523],
         [-0.209, -0.523,  1.000]],

        [[ 1.000,  0.050,  0.226],
         [ 0.050,  1.000,  0.530],
         [ 0.226,  0.530,  1.000]],

        [[ 1.000, -0.230, -0.130],
         [-0.230,  1.000,  0.131],
         [-0.130,  0.131,  1.000]],

        [[ 1.000,  0.405,  0.501],
         [ 0.405,  1.000,  0.802],
         [ 0.501,  0.802,  1.000]],

        [[ 1.000,  0.540, -0.457],
         [ 0.540,  1.000, -0.056],
         [-0.457, -0.056,  1.000]],

        [[ 1.000, -0.038,  0.850],
         [-0.038,  1.000,  0.383],
         [ 0.850,  0.383,  1.000]],

        [[ 1.000,  0.503,  0.335],
         [ 0.503,  1.000,  0.294],
         [ 0.335,  0.294,  1.000]],

        [[ 1.000,  0.486,  0.225],
  

# Load model

In [66]:
from recbole_cdr.model.layers import Binarize
binary = Binarize.apply
emb = torch.cat([
    torch.ones(2, 3, 3, requires_grad=True),
    torch.ones(2, 3, 3, requires_grad=True),
])
mask = torch.tensor([-0.1, -1, 0.1, 1], requires_grad=True)
print(mask)
loss = F.relu((binary(mask).unsqueeze(1).unsqueeze(1) * emb)).sum()
print(loss)
loss.backward()
print(mask.grad)

tensor([-0.100, -1.000,  0.100,  1.000], requires_grad=True)
tensor(18., grad_fn=<SumBackward0>)
tensor([0., 0., 9., 9.])


In [1]:
import torch
import torch.nn.functional as F
from recbole_cdr.model.layers import Binarize
binary = Binarize.apply

path = 'recbole-cdr-ckpt/MacridDAE36-Aug-03-2023_10-25-21.313869.pth'
file = torch.load(path, map_location='cpu')
para = file['state_dict']
torch.set_printoptions(precision=3,sci_mode=False)
L = para['cores_logvar.weight']
L = torch.tril(L)
L = F.normalize(L)
print(L @ L.T)
print(para['mask_source.weight'])
print(para['mask_target.weight'])
print(binary(para['mask_source.weight']))
print(binary(para['mask_target.weight']))

/home/mjyin/miniconda3/envs/recbole-cdr/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ 1.000, -0.990, -0.313, -0.320, -0.384, -0.287, -0.228, -0.299, -0.266,
         -0.268],
        [-0.990,  1.000,  0.330,  0.366,  0.433,  0.327,  0.278,  0.344,  0.317,
          0.315],
        [-0.313,  0.330,  1.000,  0.921,  0.792,  0.746,  0.698,  0.632,  0.585,
          0.534],
        [-0.320,  0.366,  0.921,  1.000,  0.904,  0.839,  0.806,  0.758,  0.723,
          0.665],
        [-0.384,  0.433,  0.792,  0.904,  1.000,  0.926,  0.886,  0.845,  0.815,
          0.759],
        [-0.287,  0.327,  0.746,  0.839,  0.926,  1.000,  0.965,  0.911,  0.878,
          0.817],
        [-0.228,  0.278,  0.698,  0.806,  0.886,  0.965,  1.000,  0.953,  0.916,
          0.866],
        [-0.299,  0.344,  0.632,  0.758,  0.845,  0.911,  0.953,  1.000,  0.961,
          0.921],
        [-0.266,  0.317,  0.585,  0.723,  0.815,  0.878,  0.916,  0.961,  1.000,
          0.961],
        [-0.268,  0.315,  0.534,  0.665,  0.759,  0.817,  0.866,  0.921,  0.961,
          1.000]])
tensor([[

In [6]:
torch.linalg.eigvals(L @ L.T)

tensor([     0.018+0.j,      0.011+0.j,      0.004+0.j,      0.002+0.j,      0.001+0.j,
             0.001+0.j,      0.000+0.j,      0.000+0.j,     -0.000+0.j,      0.000+0.j])

In [103]:
F.normalize(torch.ones(5, 5))

tensor([[0.4472, 0.4472, 0.4472, 0.4472, 0.4472],
        [0.4472, 0.4472, 0.4472, 0.4472, 0.4472],
        [0.4472, 0.4472, 0.4472, 0.4472, 0.4472],
        [0.4472, 0.4472, 0.4472, 0.4472, 0.4472],
        [0.4472, 0.4472, 0.4472, 0.4472, 0.4472]])

In [31]:
para['intent_graph']

tensor([[0.4556, 0.0780, 0.0832, 0.0215, 0.0351, 0.0583, 0.0557, 0.0616, 0.0282,
         0.0680],
        [0.0780, 0.4302, 0.0407, 0.0546, 0.1015, 0.0581, 0.0413, 0.0711, 0.0675,
         0.0573],
        [0.0832, 0.0407, 0.4533, 0.0374, 0.0638, 0.0650, 0.0538, 0.0776, 0.0198,
         0.0235],
        [0.0215, 0.0546, 0.0374, 0.4524, 0.0711, 0.0576, 0.0436, 0.0835, 0.0370,
         0.0423],
        [0.0351, 0.1015, 0.0638, 0.0711, 0.4420, 0.0649, 0.0255, 0.0500, 0.0497,
         0.0616],
        [0.0583, 0.0581, 0.0650, 0.0576, 0.0649, 0.4673, 0.0432, 0.0685, 0.0474,
         0.0514],
        [0.0557, 0.0413, 0.0538, 0.0436, 0.0255, 0.0432, 0.4598, 0.0493, 0.0757,
         0.0599],
        [0.0616, 0.0711, 0.0776, 0.0835, 0.0500, 0.0685, 0.0493, 0.4121, 0.0802,
         0.0642],
        [0.0282, 0.0675, 0.0198, 0.0370, 0.0497, 0.0474, 0.0757, 0.0802, 0.4713,
         0.0540],
        [0.0680, 0.0573, 0.0235, 0.0423, 0.0616, 0.0514, 0.0599, 0.0642, 0.0540,
         0.4601]])

In [11]:
cores = F.normalize(para['k_embedding.weight'])

In [7]:
cores @ cores.T

tensor([[ 1.0000, -0.2061,  0.1017, -0.0519, -0.1014,  0.1964, -0.0731,  0.0071,
          0.0332, -0.2584],
        [-0.2061,  1.0000, -0.1510, -0.0365,  0.0621, -0.3583,  0.0017,  0.0953,
          0.0277,  0.1007],
        [ 0.1017, -0.1510,  1.0000, -0.0960,  0.1342,  0.0763,  0.0417,  0.0913,
         -0.1019,  0.0535],
        [-0.0519, -0.0365, -0.0960,  1.0000, -0.0736,  0.1676,  0.2307, -0.1100,
          0.0150, -0.0579],
        [-0.1014,  0.0621,  0.1342, -0.0736,  1.0000,  0.1688, -0.0911,  0.0505,
         -0.0730,  0.1633],
        [ 0.1964, -0.3583,  0.0763,  0.1676,  0.1688,  1.0000,  0.0529,  0.0302,
         -0.0715,  0.1613],
        [-0.0731,  0.0017,  0.0417,  0.2307, -0.0911,  0.0529,  1.0000,  0.0203,
         -0.1387,  0.0125],
        [ 0.0071,  0.0953,  0.0913, -0.1100,  0.0505,  0.0302,  0.0203,  1.0000,
          0.2779,  0.0314],
        [ 0.0332,  0.0277, -0.1019,  0.0150, -0.0730, -0.0715, -0.1387,  0.2779,
          1.0000, -0.2568],
        [-0.2584,  

In [ ]:
mask = para['mask_source.weight']
mask

In [ ]:
mask = para['mask_target.weight']
mask

In [ ]:
mask_source = (para['mask_source.weight'] > 0)
mask_source

In [ ]:
mask_target = (para['mask_target.weight'] > 0)
mask_target

In [ ]:
(~(mask_source | mask_target)).sum()

In [ ]:
items = F.normalize(para['item_embedding.weight'], dim=1)
cores = F.normalize(para['k_embedding.weight'], dim=1)
cates_logits = torch.matmul(items, cores.transpose(0, 1)) / 0.2

In [ ]:
val, idx = F.softmax(cates_logits, dim=-1).max(-1)

In [ ]:
data = torch.randn(1, 5, requires_grad=True)
data

In [ ]:
F.gumbel_softmax(data, hard=False, tau=1)

In [ ]:
from recbole_cdr.model.layers import Binarize
binary = Binarize.apply

In [ ]:
mask1 = nn.Embedding(1, 4)
mask2 = nn.Embedding(1, 4)
mask1.weight.data.copy_(torch.tensor([0.2, 0.5, 0.7, -0.5]))
mask2.weight.data.copy_(torch.tensor([-0.4, 0.2, 0.6, -0.3]))
net = nn.ModuleList([mask1, mask2])
from torch.optim import Adam
opt = Adam(net.parameters(), lr=0.1)

In [ ]:
b_mask1, b_mask2 = binary(mask1.weight), binary(mask2.weight)
loss = (F.relu(1 - (b_mask1 + b_mask2))).sum()
loss

In [ ]:
opt.zero_grad()
loss.backward()
opt.step()

In [ ]:
mask1.weight

In [ ]:
mask2.weight